In [1]:
import time
!pip install ultralytics

In [2]:
!pip install -U ipywidgets

^C
ERROR: Operation cancelled by user


In [1]:
from ultralytics import YOLO

#### Training the model

In [ ]:
results = model.train(
    data='/kaggle/input/80-10-10-dataset/data.yaml',
    epochs=100,
    imgsz=640,
    patience=50,
    device=0,
    batch=64,
    optimizer='SGD',
    momentum=0.95
)

#### Evaluation

In [ ]:
model = YOLO("best_80_10_10.pt")

metrics = model.val()
metrics.box.map
metrics.box.map50
metrics.box.map75
metrics.box.maps

#### hyperparameter-tuning

- in our case SGD as optimizer is better that AdamW


In [ ]:
model = YOLO("best_s_version.pt")

- lr0 = [0.0005, **0.001**, 0.005]
- lrf = [0.01, **0.005**]
- batch = [16, **32**, 64]
- momentum = [0.85, **0.9**, 0.95]
- warmup_momentum = [**0.5**]
- warmup_bias_lr = [**0.05**, 0.1, 0.2]
- weight_decay = [0.0001, **0.0005**, 0.001]
- patience = [**30**, 50]
- imgsz = [**640**, 800, 1024]


In [ ]:
results = model.train(
    data='/kaggle/input/80-10-10-dataset/data.yaml',
    epochs=50,
    imgsz=640,
    patience=30,
    device=0,
    batch=32,
    optimizer='SGD',
    lr0=0.001,
    lrf=0.005,
    momentum=0.9,
    warmup_momentum=0.5,
    warmup_bias_lr=0.05,
    weight_decay=0.0005,
)


#### Testing the model

In [1]:
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator
import cv2

In [3]:
model = YOLO("best_s_version.pt")
video_path = 'sample_02.mp4'
cap = cv2.VideoCapture(video_path)

FileNotFoundError: [Errno 2] No such file or directory: 'best_s_version.pt'

In [8]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)

    for r in results:
        annotator = Annotator(frame)

        boxes = r.boxes
        for box in boxes:
            b = box.xyxy[0]
            c = box.cls
            annotator.box_label(b, model.names[int(c)])

    frame = annotator.result()
    cv2.imshow("Object Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

#### Alarm Testing

In [4]:
!pip install pydub pyaudio

  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached PyAudio-0.2.14.tar.gz (47 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)
  Created wheel for pyaudio: filename=PyAudio-0.2.14-cp39-cp39-linux_x86_64.whl size=27654 sha256=d8c2d585db6ba3d18885751dbc67334b1d66c3928933202efb1d08a6d1062339
  Stored in directory: /home/mo/.cache/pip/wheels/28/d3/62/6ad369dc09fe82e1c9ceb83601a800eb305b901df7789aa550
Successfully built pyaudio


In [2]:
from alarm import AlarmDetector
from pydub import AudioSegment
from pydub.playback import play

/home/mo/miniconda3/envs/yolo/lib/python3.9/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [21]:
import time

model = YOLO("../models/best_s_version.pt")
video_path = '../video_samples/sample_01.mp4'
alarm_sound = AudioSegment.from_file('alarm.wav')
cap = cv2.VideoCapture(video_path)
calc = AlarmDetector(model, 300, 50)
alarm = False

while True:
    ret, frame = cap.read(10)
    if not ret:
        break

    results = model(frame, stream=True)

    for r in results:
        time.sleep(.01)
        annotator = Annotator(frame)

        boxes = r.boxes
        calc.add_boxes(boxes)
        for box in boxes:
            b = box.xyxy[0]
            c = box.cls
            annotator.box_label(b, model.names[int(c)])

        if calc.check_alarm() and not alarm:
            alarm = True
            play(alarm_sound)
            # add to database and send notifications

    frame = annotator.result()
    cv2.imshow("Object Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 640x384 (no detections), 10.2ms
Speed: 2.4ms preprocess, 10.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 10.8ms
Speed: 3.0ms preprocess, 10.8ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 Drowning, 9.0ms
Drowning
Speed: 2.4ms preprocess, 9.0ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 Drowning, 1 Swimming, 12.2ms
Drowning
Speed: 2.5ms preprocess, 12.2ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 Drowning, 9.2ms
Drowning
Speed: 3.3ms preprocess, 9.2ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 Drownings, 9.9ms
Drowning
Speed: 2.6ms preprocess, 9.9ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 Drowning, 1 Swimming, 13.2ms
Drowning
Speed: 2.8ms preprocess, 13.2ms inference, 2.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 Drowning, 8.6ms


ALSA lib pcm_dsnoop.c:601:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1032:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'
ALSA lib pcm_dmix.c:1032:(snd_pcm_dmix_open) unable to open slave


Speed: 3.7ms preprocess, 10.8ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 Drowning, 10.8ms
Drowning
Speed: 3.1ms preprocess, 10.8ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 Drowning, 10.9ms
Drowning
Speed: 3.0ms preprocess, 10.9ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 Drowning, 10.8ms
Drowning
Speed: 3.1ms preprocess, 10.8ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 Drowning, 11.1ms
Drowning
Speed: 3.2ms preprocess, 11.1ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 Drowning, 11.7ms
Drowning
Speed: 3.2ms preprocess, 11.7ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 Drowning, 7.0ms
Drowning
Speed: 2.6ms preprocess, 7.0ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 Drowning, 6.9ms
Drowning
Speed: 2.5ms preprocess, 6.9ms inference, 1.

In [6]:
cap.release()
cv2.destroyAllWindows()

#### activate the Tracker in Yolov8